# Real-World Crime Analytics for the London Metropolitan Area

## 1. Environment Setup (Linux)

### 1.1. Update Linux Packages

In [ ]:
!sudo apt update

### 1.2. Install Python and Packages (pip & venv)

In [ ]:
!sudo apt install -y python3 python3-pip python3-venv

### 1.3. Create a Virtual Environment

In [ ]:
!python3 -m venv venv

### 1.4. Activate the Virtual Environment

In [ ]:
!source venv/bin/activate

### 1.5. Install the Required Python Packages

In [ ]:
%pip install -r requirements.txt

## 2. Data Ingestion

### 2.1. Create Folders to Store Data

In [2]:
import os

# Create datasets folder if it doesn't exist
os.makedirs("Datasets", exist_ok=True)
os.makedirs("Datasets/Raw-Data", exist_ok=True)
os.makedirs("Datasets/Raw-Data/Map-Data", exist_ok=True)
os.makedirs("Datasets/Raw-Data/Police-Data", exist_ok=True)
os.makedirs("Datasets/Raw-Data/Income-Data", exist_ok=True)
os.makedirs("Datasets/Raw-Data/Population-Data", exist_ok=True)

### 2.2. Download Data

#### 2.2.1. Download Income Data - <a>www.ons.gov.uk</a>

In [ ]:
!wget -O Datasets/Raw-Data/Income-Data/Income-MSOA.xlsx "https://www.ons.gov.uk/file?uri=/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/smallareaincomeestimatesformiddlelayersuperoutputareasenglandandwales/financialyearending2020/saiefy1920finalqaddownload280923.xlsx"
print("Downloaded to Datasets/Raw-Data/Income-Data/Income-MSOA.xlsx")

#### 2.2.2. Download Population Data - <a>www.ons.gov.uk</a>

In [ ]:
!wget -O Datasets/Raw-Data/Population-Data/Population-LSOA.xlsx "https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/lowersuperoutputareamidyearpopulationestimates/mid2022revisednov2025tomid2024/sapelsoasyoa20222024.xlsx"
print("Downloaded to Datasets/Raw-Data/Population-Data/Population-LSOA.xlsx")

#### 2.2.3. Download Police Data - <a>data.police.uk</a>

In [ ]:
!wget -O Datasets/Raw-Data/Police-Data/RAW-POLICE-2022-10-TO-2025-09.zip "https://data.police.uk/data/archive/2025-09.zip"
print("Downloaded to Datasets/Raw-Data/RAW-POLICE-2022-10-TO-2025-09.zip")

In [ ]:
# Extract the police data zip file
!unzip -o Datasets/Raw-Data/RAW-POLICE-2022-10-TO-2025-09.zip -d Datasets/Raw-Data/Police-Data
print("Extracted police data to Datasets/Raw-Data/Police-Data/")

In [ ]:
# Clean up the zip file to save space
!rm -rf Datasets/Raw-Data/RAW-POLICE-2022-10-TO-2025-09.zip

#### 2.2.4. Download Map Data - <a>geoportal.statistics.gov.uk</a>

#### Note: Map Data from ONS Geography Can't be Downloaded Directly, So Use the Below Links to Download Them.

1. LSOAs - <a>https://geoportal.statistics.gov.uk/datasets/ons::output-areas-december-2021-boundaries-ew-bgc-v2/about</a>

2. MSOAs - <a>https://geoportal.statistics.gov.uk/datasets/ons::middle-layer-super-output-areas-december-2021-boundaries-ew-bfc-v7-2/about</a>

### 2.3. Converting File Types and Keeping Necessary Files

#### 2.3.1. Convert XLSX (Excel) files to CSV using pandas

In [ ]:
import pandas as pd
import os
import re

# Safe filename from sheet name
def sanitize(name: str) -> str:
    # Replace non-alphanumeric with underscores, strip, collapse repeats
    name = re.sub(r"[^A-Za-z0-9]+", "_", name).strip("_")
    return re.sub(r"_+", "_", name) or "Sheet"

# Convert Income-MSOA.xlsx: write one CSV per sheet
income_xlsx_path = "Datasets/Raw-Data/Income-Data/Income-MSOA.xlsx"
income_out_dir = "Datasets/Raw-Data/Income-Data/Income-MSOA"
os.makedirs(income_out_dir, exist_ok=True)

try:
    xls_income = pd.ExcelFile(income_xlsx_path)
    for sheet in xls_income.sheet_names:
        df = pd.read_excel(xls_income, sheet_name=sheet)
        safe = sanitize(sheet)
        out_path = os.path.join(income_out_dir, f"Income-MSOA-{safe}.csv")
        df.to_csv(out_path, index=False)
    print(f"Exported {len(xls_income.sheet_names)} sheets from {income_xlsx_path} to {income_out_dir}")
except Exception as e:
    print("Failed to process income workbook:", e)

# Convert Population-LSOA.xlsx: write one CSV per sheet
population_xlsx_path = "Datasets/Raw-Data/Population-Data/Population-LSOA.xlsx"
population_out_dir = "Datasets/Raw-Data/Population-Data/Population-LSOA"
os.makedirs(population_out_dir, exist_ok=True)

try:
    xls_pop = pd.ExcelFile(population_xlsx_path)
    for sheet in xls_pop.sheet_names:
        df = pd.read_excel(xls_pop, sheet_name=sheet)
        safe = sanitize(sheet)
        out_path = os.path.join(population_out_dir, f"Population-LSOA-{safe}.csv")
        df.to_csv(out_path, index=False)
    print(f"Exported {len(xls_pop.sheet_names)} sheets from {population_xlsx_path} to {population_out_dir}")
except Exception as e:
    print("Failed to process population workbook:", e)

#### 2.3.2. Retain Only Metropolitan Police Records

In [ ]:
from pathlib import Path
from typing import List

police_root = Path("Datasets/Raw-Data/Police-Data")
date_range = ("2022-10", "2025-09")
keep_token = "metropolitan"

if not police_root.exists():
    raise FileNotFoundError(f"Missing directory: {police_root}")

kept: List[Path] = []
removed: List[Path] = []

# Delete every police file that does not belong to the Metropolitan force.
for file_path in police_root.rglob("*"):
    if not file_path.is_file():
        continue
    if keep_token in file_path.name.lower():
        kept.append(file_path)
        continue
    file_path.unlink()
    removed.append(file_path)

print(f"Date Range - {date_range[0]} - {date_range[1]}")
for path in sorted(kept):
    print(path)

print(f"Removed {len(removed)} other files.")

### 2.4. Copy Only the Necessary Files for Cleaning

In [ ]:
os.makedirs("Datasets/Data-for-Cleaning", exist_ok=True)
os.makedirs("Datasets/Data-for-Cleaning/Police-Data", exist_ok=True)
os.makedirs("Datasets/Data-for-Cleaning/Income-Data", exist_ok=True)
os.makedirs("Datasets/Data-for-Cleaning/Population-Data", exist_ok=True)
os.makedirs("Datasets/Data-for-Cleaning/Map-Data", exist_ok=True)

#### 2.4.1. Copy Police Data

In [ ]:
!cp -r Datasets/Raw-Data/Police-Data/* Datasets/Data-for-Cleaning/Police-Data/

#### 2.4.2. Copy Map Data

In [ ]:
!cp -r Datasets/Raw-Data/Map-Data/* Datasets/Data-for-Cleaning/Map-Data/

#### 2.4.3. Copy Income Data

In [ ]:
!cp -r Datasets/Raw-Data/Income-Data/Income-MSOA/Income-MSOA-Total_annual_income.csv Datasets/Data-for-Cleaning/Income-Data/

#### 2.4.4. Copy Population Data

In [ ]:
!cp -r Datasets/Raw-Data/Population-Data/Population-LSOA/Population-LSOA-Mid_2022_LSOA_2021.csv Datasets/Data-for-Cleaning/Population-Data/
!cp -r Datasets/Raw-Data/Population-Data/Population-LSOA/Population-LSOA-Mid_2023_LSOA_2021.csv Datasets/Data-for-Cleaning/Population-Data/
!cp -r Datasets/Raw-Data/Population-Data/Population-LSOA/Population-LSOA-Mid_2024_LSOA_2021.csv Datasets/Data-for-Cleaning/Population-Data/

## 3. Data Cleaning

### 3. 1. Initialize Spark

In [2]:
# Import and initialize Spark
import pyspark
from pathlib import Path
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import NumericType


# Initialize session
spark = SparkSession.builder \
    .appName("Real-World Crime Analytics for the London Metropolitan Area") \
    .master("local[*]") \
    .getOrCreate()

print("Spark version:", spark.version)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/10 10:59:21 WARN Utils: Your hostname, codespaces-044927, resolves to a loopback address: 127.0.0.1; using 10.0.4.134 instead (on interface eth0)
25/12/10 10:59:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/10 10:59:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 4.0.1


### 3. 2. Cleaning Income Data

#### 3.2.1.  Reading the Income Data - Data & Schema

In [3]:
# Load and display the total annual income data
total_income_path = "Datasets/Data-for-Cleaning/Income-Data/Income-MSOA-Total_annual_income.csv"

total_income_df = spark.read.option("header", "true").csv(total_income_path)

total_income_df.show(5)
total_income_df.printSchema()

+--------------------+-----------------+--------------------+--------------------+-----------+-----------+--------------------+--------------------+--------------------+--------------------+
|            Contents|       Unnamed: 1|          Unnamed: 2|          Unnamed: 3| Unnamed: 4| Unnamed: 5|          Unnamed: 6|          Unnamed: 7|          Unnamed: 8|          Unnamed: 9|
+--------------------+-----------------+--------------------+--------------------+-----------+-----------+--------------------+--------------------+--------------------+--------------------+
|                NULL|             NULL|                NULL|                NULL|       NULL|       NULL|                NULL|                NULL|                NULL|                NULL|
|Total annual hous...|             NULL|                NULL|                NULL|       NULL|       NULL|                NULL|                NULL|                NULL|                NULL|
|                NULL|             NULL|     

#### 3.2.2.  Remove Extra Header Lines and Load Cleaned Income Data

In [4]:
# read as text so we can inspect the actual first lines
raw = spark.read.text(total_income_path)

# Drop the first 4 lines
clean_lines = raw.rdd.zipWithIndex() \
    .filter(lambda x: x[1] >= 4) \
    .map(lambda x: x[0].value)

# Convert back to CSV DataFrame
clean_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(clean_lines)

total_income_df = clean_df

total_income_df.show(5)
total_income_df.printSchema()

+---------+-----------------+--------------------+--------------------+-----------+-----------+-----------------------+--------------------------+--------------------------+-----------------------+
|MSOA code|        MSOA name|Local authority code|Local authority name|Region code|Region name|Total annual income (£)|Upper confidence limit (£)|Lower confidence limit (£)|Confidence interval (£)|
+---------+-----------------+--------------------+--------------------+-----------+-----------+-----------------------+--------------------------+--------------------------+-----------------------+
|E02004297|County Durham 001|           E06000047|       County Durham|  E12000001| North East|                  41400|                     49100|                     34900|                  14200|
|E02004290|County Durham 002|           E06000047|       County Durham|  E12000001| North East|                  41100|                     48800|                     34700|                  14100|
|E02004298

#### 3.2.3. Summary Statistics for Cleaned Income Data

In [5]:
total_income_df.describe().show()

25/12/10 10:59:32 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+---------+---------+--------------------+--------------------+-----------+-----------+-----------------------+--------------------------+--------------------------+-----------------------+
|summary|MSOA code|MSOA name|Local authority code|Local authority name|Region code|Region name|Total annual income (£)|Upper confidence limit (£)|Lower confidence limit (£)|Confidence interval (£)|
+-------+---------+---------+--------------------+--------------------+-----------+-----------+-----------------------+--------------------------+--------------------------+-----------------------+
|  count|     7201|     7201|                7201|                7201|       7201|       7201|                   7201|                      7201|                      7201|                   7201|
|   mean|     NULL|     NULL|                NULL|                NULL|       NULL|       NULL|      46445.54922927371|         55075.72559366754|        39170.143035689485|      15905.58255797806|
| stddev| 

#### 3.2.4. Detecting Duplicate Rows in Cleaned Income Data


In [6]:
duplicates = total_income_df.groupBy(total_income_df.columns) \
               .count() \
               .filter("count > 1")

duplicates.show()

+---------+---------+--------------------+--------------------+-----------+-----------+-----------------------+--------------------------+--------------------------+-----------------------+-----+
|MSOA code|MSOA name|Local authority code|Local authority name|Region code|Region name|Total annual income (£)|Upper confidence limit (£)|Lower confidence limit (£)|Confidence interval (£)|count|
+---------+---------+--------------------+--------------------+-----------+-----------+-----------------------+--------------------------+--------------------------+-----------------------+-----+
+---------+---------+--------------------+--------------------+-----------+-----------+-----------------------+--------------------------+--------------------------+-----------------------+-----+



### 3. 3. Cleaning Population Data

#### 3.3.1.  Reading the Population Data - Data & Schema

In [7]:
population_paths = {
    "2022": "Datasets/Data-for-Cleaning/Population-Data/Population-LSOA-Mid_2022_LSOA_2021.csv",
    "2023": "Datasets/Data-for-Cleaning/Population-Data/Population-LSOA-Mid_2023_LSOA_2021.csv",
    "2024": "Datasets/Data-for-Cleaning/Population-Data/Population-LSOA-Mid_2024_LSOA_2021.csv",
}

population_dfs = {}
for year, path in population_paths.items():
    df = spark.read.option("header", "true").csv(path)
    population_dfs[year] = df
    print(f"Population {year} Data:")
    df.show(5)
    df.printSchema()

population_2022_df = population_dfs["2022"]
population_2023_df = population_dfs["2023"]
population_2024_df = population_dfs["2024"]

Population 2022 Data:
+-----------------------------------------------------------------------------------------+-------------+--------------+---------------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------

#### 3.3.2.  Remove Extra Header Lines and Load Cleaned Population Data

In [8]:
def clean_population(path: str):
    rows = spark.read.text(path)
    trimmed = rows.rdd.zipWithIndex().filter(lambda x: x[1] >= 3).map(lambda x: x[0].value)
    return spark.read.option("header", "true").option("inferSchema", "true").csv(trimmed)

population_clean = {year: clean_population(path) for year, path in population_paths.items()}

population_2022_df = population_clean["2022"]
population_2023_df = population_clean["2023"]
population_2024_df = population_clean["2024"]

for year, df in population_clean.items():
    print(f"Cleaned Population {year} Data:")
    df.show(5)
    df.printSchema()

Cleaned Population 2022 Data:
+-------------+-------------+--------------+---------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|LAD 2023 Code|LAD 2023 Name|LSOA 2021 Code| LSOA 2021 Name|Total| F0| F1| F2| F3| F4| F5| F6| F7| F8| F9|F10|F11|F12|F13|F14|F15|F16|F17|F18|F19|F20|F21|F22|F23|F24|F25|F26|F

#### 3.3.3. Summary Statistics for Cleaned Population Data

In [9]:
print("Population 2022 Data:")
population_2022_df.describe().show()

print("Population 2023 Data:")
population_2023_df.describe().show()

print("Population 2024 Data:")
population_2024_df.describe().show()

Population 2022 Data:


25/12/10 11:00:00 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB


+-------+-------------+-------------+--------------+--------------+------------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-

25/12/10 11:00:15 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB


+-------+-------------+-------------+--------------+--------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+----------------

25/12/10 11:00:28 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB


+-------+-------------+-------------+--------------+--------------+------------------+-----------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------

#### 3.3.4. Detecting Duplicate Rows in Cleaned Population Data


In [10]:
duplicates = {
    year: df.groupBy(*df.columns).count().filter("count > 1")
    for year, df in population_clean.items()
}

duplicates_2022 = duplicates["2022"]
duplicates_2023 = duplicates["2023"]
duplicates_2024 = duplicates["2024"]

for year, dup_df in duplicates.items():
    print(f"Duplicate Rows in Population {year} Data:")
    dup_df.show()

Duplicate Rows in Population 2022 Data:


+-------------+-------------+--------------+--------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+
|LAD 2023 Code|LAD 2023 Name|LSOA 2021 Code|LSOA 2021 Name|Total| F0| F1| F2| F3| F4| F5| F6| F7| F8| F9|F10|F11|F12|F13|F14|F15|F16|F17|F18|F19|F20|F21|F22|F23|F24|F25|F26|F27|F28|F29|F30|F31|F32|F33

+-------------+-------------+--------------+--------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+
|LAD 2023 Code|LAD 2023 Name|LSOA 2021 Code|LSOA 2021 Name|Total| F0| F1| F2| F3| F4| F5| F6| F7| F8| F9|F10|F11|F12|F13|F14|F15|F16|F17|F18|F19|F20|F21|F22|F23|F24|F25|F26|F27|F28|F29|F30|F31|F32|F33

+-------------+-------------+--------------+--------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+
|LAD 2023 Code|LAD 2023 Name|LSOA 2021 Code|LSOA 2021 Name|Total| F0| F1| F2| F3| F4| F5| F6| F7| F8| F9|F10|F11|F12|F13|F14|F15|F16|F17|F18|F19|F20|F21|F22|F23|F24|F25|F26|F27|F28|F29|F30|F31|F32|F33

### 3. 4. Cleaning Map Data

#### 3.4.1.  Reading the Map Data - Data & Schema

In [11]:
# Load and display the population datasets
map_paths = {
    "lsoa": "Datasets/Data-for-Cleaning/Map-Data/Map-LSOA-2021.csv",
    "msoa": "Datasets/Data-for-Cleaning/Map-Data/Map-MSOA-2021.csv",
}

map_dfs = {}
for map, path in map_paths.items():
    df = spark.read.option("header", "true").csv(path)
    map_dfs[map] = df
    print(f"Population {map} Data:")
    df.show(5)
    df.printSchema()

map_lsoa_df = map_dfs["lsoa"]
map_msoa_df = map_dfs["msoa"]


Population lsoa Data:
+---+---------+---------+-------------------+---------+------+------+--------+--------+----------------+----------------+--------------------+
|FID|   OA21CD| LSOA21CD|           LSOA21NM|LSOA21NMW| BNG_E| BNG_N|     LAT|    LONG|     Shape__Area|   Shape__Length|            GlobalID|
+---+---------+---------+-------------------+---------+------+------+--------+--------+----------------+----------------+--------------------+
|  1|E00000001|E01000001|City of London 001A|         |532250|181864|51.52022|-0.09523|6949.15148162842|421.166161134957|3a44dd3d-5082-4a0...|
|  2|E00000003|E01000001|City of London 001A|         |532171|181819|51.51983|-0.09638|4492.41107177734|307.714652773369|f1216dc8-14d1-485...|
|  3|E00000005|E01000001|City of London 001A|         |532166|181722|51.51896|-0.09649|8565.51421356201|385.204781363993|44d6f70f-549c-428...|
|  4|E00000007|E01000001|City of London 001A|         |532088|181473|51.51674|-0.09771|75994.8297042847|1408.60765694562

#### 3.4.2. Summary Statistics for Map Data

In [12]:
print("Map LSOA Data:")
map_lsoa_df.describe().show()

print("Map MSOA Data:")
map_msoa_df.describe().show()


Map LSOA Data:


+-------+-----------------+---------+---------+---------+-------------+-----------------+------------------+------------------+-------------------+------------------+------------------+--------------------+
|summary|              FID|   OA21CD| LSOA21CD| LSOA21NM|    LSOA21NMW|            BNG_E|             BNG_N|               LAT|               LONG|       Shape__Area|     Shape__Length|            GlobalID|
+-------+-----------------+---------+---------+---------+-------------+-----------------+------------------+------------------+-------------------+------------------+------------------+--------------------+
|  count|           188880|   188880|   188880|   188880|       188880|           188880|            188880|            188880|             188880|            188880|            188880|              188880|
|   mean|          94440.5|     NULL|     NULL|     NULL|         NULL|446380.5484540449| 273724.4691444303|52.352316229828475|-1.3269725055326131|  800720.004194128|2992.1

#### 3.4.3. Detecting Duplicate Rows in Map Data


In [13]:
duplicates_map = {
    "lsoa": map_lsoa_df.groupBy(*map_lsoa_df.columns).count().filter("count > 1"),
    "msoa": map_msoa_df.groupBy(*map_msoa_df.columns).count().filter("count > 1"),
}

duplicates_lsoa = duplicates_map["lsoa"]
duplicates_msoa = duplicates_map["msoa"]

for map_type, dup_df in duplicates_map.items():
    print(f"Duplicate Rows in Map {map_type.upper()} Data:")
    dup_df.show()

Duplicate Rows in Map LSOA Data:


+---+------+--------+--------+---------+-----+-----+---+----+-----------+-------------+--------+-----+
|FID|OA21CD|LSOA21CD|LSOA21NM|LSOA21NMW|BNG_E|BNG_N|LAT|LONG|Shape__Area|Shape__Length|GlobalID|count|
+---+------+--------+--------+---------+-----+-----+---+----+-----------+-------------+--------+-----+
+---+------+--------+--------+---------+-----+-----+---+----+-----------+-------------+--------+-----+

Duplicate Rows in Map MSOA Data:
+---+--------+--------+---------+-----+-----+---+----+-----------+-------------+--------+-----+
|FID|MSOA21CD|MSOA21NM|MSOA21NMW|BNG_E|BNG_N|LAT|LONG|Shape__Area|Shape__Length|GlobalID|count|
+---+--------+--------+---------+-----+-----+---+----+-----------+-------------+--------+-----+
+---+--------+--------+---------+-----+-----+---+----+-----------+-------------+--------+-----+



### 3. 5. Cleaning Police Data

#### 3.5.1. Combine All Police data into a Single Frame


In [14]:

police_data_root = Path("Datasets/Data-for-Cleaning/Police-Data")

def load_police_dataset(file_glob: str) -> DataFrame:
    """Load all monthly police CSVs matching the glob into a single DataFrame."""
    # Gather every CSV matching the pattern across the month folders.
    matches = sorted(police_data_root.rglob(file_glob))
    if not matches:
        raise FileNotFoundError(f"No police files matched pattern: {file_glob}")
    print(f"Matched {len(matches)} files for pattern '{file_glob}'")
    # Load all matched files into a single Spark DataFrame.
    df = spark.read.option("header", "true").option("inferSchema", "true").csv([str(path) for path in matches])

    return df

# Build combined DataFrames for each police dataset family.
police_outcomes_df = load_police_dataset("*-metropolitan-outcomes.csv")
police_stop_search_df = load_police_dataset("*-metropolitan-stop-and-search.csv")
police_street_df = load_police_dataset("*-metropolitan-street.csv")

# Register temp views for downstream Spark SQL operations.
police_outcomes_df.createOrReplaceTempView("police_outcomes")
police_stop_search_df.createOrReplaceTempView("police_stop_and_search")
police_street_df.createOrReplaceTempView("police_street")

# Print simple row counts for a quick sanity check.
print("Combined police outcomes rows:", police_outcomes_df.count())
print("Combined police stop and search rows:", police_stop_search_df.count())
print("Combined police street rows:", police_street_df.count())

Matched 36 files for pattern '*-metropolitan-outcomes.csv'


Matched 36 files for pattern '*-metropolitan-stop-and-search.csv'


Matched 36 files for pattern '*-metropolitan-street.csv'


Combined police outcomes rows: 2794424


Combined police stop and search rows: 402848


Combined police street rows: 3415444


#### 3.5.2. Police Outcomes Data Check

##### 3.5.2.1. Police Outcomes Schema Check

In [15]:
print("Police Outcomes Schema:")
police_outcomes_df.printSchema()

Police Outcomes Schema:
root
 |-- Crime ID: string (nullable = true)
 |-- Month: timestamp (nullable = true)
 |-- Reported by: string (nullable = true)
 |-- Falls within: string (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- LSOA code: string (nullable = true)
 |-- LSOA name: string (nullable = true)
 |-- Outcome type: string (nullable = true)



##### 3.5.2.2. Police Outcomes Data Rows Check

In [16]:
print("First 5 Records")
police_outcomes_df.orderBy("Month").show(5)

print("Last 5 Records")
police_outcomes_df.orderBy("Month").tail(5)

First 5 Records


+--------------------+-------------------+--------------------+--------------------+---------+--------+-----------+---------+----------------+--------------------+
|            Crime ID|              Month|         Reported by|        Falls within|Longitude|Latitude|   Location|LSOA code|       LSOA name|        Outcome type|
+--------------------+-------------------+--------------------+--------------------+---------+--------+-----------+---------+----------------+--------------------+
|781de9db783d5ab46...|2022-10-01 00:00:00|Metropolitan Poli...|Metropolitan Poli...|     NULL|    NULL|No location|     NULL|            NULL|Investigation com...|
|40d4690686e6222e9...|2022-10-01 00:00:00|Metropolitan Poli...|Metropolitan Poli...|     NULL|    NULL|No location|     NULL|            NULL|Investigation com...|
|edc7a8f5f85566d53...|2022-10-01 00:00:00|Metropolitan Poli...|Metropolitan Poli...|     NULL|    NULL|No location|     NULL|            NULL|    Local resolution|
|13c5599382e8940

[Row(Crime ID='2081d3d7fbd72d83667d2cd3b241333f8c8e291e12ebbc3c22135b6a898bde48', Month=datetime.datetime(2025, 9, 1, 0, 0), Reported by='Metropolitan Police Service', Falls within='Metropolitan Police Service', Longitude=-0.481707, Latitude=51.54761, Location='On or near Parking Area', LSOA code='E01002521', LSOA name='Hillingdon 016C', Outcome type='Investigation complete; no suspect identified'),
 Row(Crime ID='5b34d94216690bc1202abb44c92786e5b0c7da666341eab49b10fa1c8df9eb18', Month=datetime.datetime(2025, 9, 1, 0, 0), Reported by='Metropolitan Police Service', Falls within='Metropolitan Police Service', Longitude=-0.139391, Latitude=51.493025, Location='On or near Upper Tachbrook Street', LSOA code='E01004749', LSOA name='Westminster 022B', Outcome type='Investigation complete; no suspect identified'),
 Row(Crime ID='5746be5b910729c6f4682d3d62de2213b26de9612721f625852f5de9496f6a5d', Month=datetime.datetime(2025, 9, 1, 0, 0), Reported by='Metropolitan Police Service', Falls within='

##### 3.5.2.3. Police Outcomes Values Check

In [17]:
police_outcomes_df.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------+---------+--------------------+
|summary|            Crime ID|         Reported by|        Falls within|           Longitude|           Latitude|            Location|LSOA code|LSOA name|        Outcome type|
+-------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------+---------+--------------------+
|  count|             2794424|             2794424|             2794424|             2760870|            2760870|             2794424|  2760869|  2760869|             2794424|
|   mean|                NULL|                NULL|                NULL|-0.11884484811345974|  51.51170855053989|                NULL|     NULL|     NULL|                NULL|
| stddev|                NULL|                NULL|                NULL| 0.15286136988882676|0.07715594243938535|       

##### 3.5.2.4. Fill missing coordinates with readable placeholders.


In [18]:
# Fill all missing values in police_outcomes_df with None
police_outcomes_df = police_outcomes_df.fillna("None")

# Detect numeric columns automatically
numeric_cols_outcomes = [
    f.name for f in police_outcomes_df.schema.fields
    if isinstance(f.dataType, NumericType)
]

# Fill all numeric columns with 0.0
if numeric_cols_outcomes:
    police_outcomes_df = police_outcomes_df.fillna(0.0, subset=numeric_cols_outcomes)

police_outcomes_df.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+---------+---------+--------------------+
|summary|            Crime ID|         Reported by|        Falls within|           Longitude|          Latitude|            Location|LSOA code|LSOA name|        Outcome type|
+-------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+---------+---------+--------------------+
|  count|             2794424|             2794424|             2794424|             2794424|           2794424|             2794424|  2794424|  2794424|             2794424|
|   mean|                NULL|                NULL|                NULL|-0.11741782056373966|50.893182561389786|                NULL|     NULL|     NULL|                NULL|
| stddev|                NULL|                NULL|                NULL|  0.1524912547801669| 5.611118329534007|             

#### 3.5.3. Police Stop and Search Data Check

##### 3.5.3.1. Police Stop and Search Schema Check

In [19]:
print("Police Stop and Search Schema:")
police_stop_search_df.printSchema()

Police Stop and Search Schema:
root
 |-- Type: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Part of a policing operation: boolean (nullable = true)
 |-- Policing operation: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age range: string (nullable = true)
 |-- Self-defined ethnicity: string (nullable = true)
 |-- Officer-defined ethnicity: string (nullable = true)
 |-- Legislation: string (nullable = true)
 |-- Object of search: string (nullable = true)
 |-- Outcome: string (nullable = true)
 |-- Outcome linked to object of search: boolean (nullable = true)
 |-- Removal of more than just outer clothing: boolean (nullable = true)



##### 3.5.3.2. Police Stop and Search Data Rows Check

In [20]:
print("First 5 Records")
police_stop_search_df.orderBy("Date").show(5)

print("Last 5 Records")
police_stop_search_df.orderBy("Date").tail(5)

First 5 Records


+--------------------+-------------------+----------------------------+------------------+---------+---------+------+---------+----------------------+-------------------------+--------------------+----------------+--------------------+----------------------------------+----------------------------------------+
|                Type|               Date|Part of a policing operation|Policing operation| Latitude|Longitude|Gender|Age range|Self-defined ethnicity|Officer-defined ethnicity|         Legislation|Object of search|             Outcome|Outcome linked to object of search|Removal of more than just outer clothing|
+--------------------+-------------------+----------------------------+------------------+---------+---------+------+---------+----------------------+-------------------------+--------------------+----------------+--------------------+----------------------------------+----------------------------------------+
|       Person search|2022-09-30 23:00:00|                      

[Row(Type='Person search', Date=datetime.datetime(2025, 9, 30, 22, 15), Part of a policing operation=False, Policing operation=None, Latitude=51.458882, Longitude=0.00448, Gender='Male', Age range='over 34', Self-defined ethnicity='White - English/Welsh/Scottish/Northern Irish/British', Officer-defined ethnicity='White', Legislation='Police and Criminal Evidence Act 1984 (section 1)', Object of search='Stolen goods', Outcome='Arrest', Outcome linked to object of search=False, Removal of more than just outer clothing=False),
 Row(Type='Person search', Date=datetime.datetime(2025, 9, 30, 22, 33), Part of a policing operation=False, Policing operation=None, Latitude=51.566148, Longitude=-0.020977, Gender='Male', Age range='25-34', Self-defined ethnicity='Black/African/Caribbean/Black British - Any other Black/African/Caribbean background', Officer-defined ethnicity='Black', Legislation='Police and Criminal Evidence Act 1984 (section 1)', Object of search='Evidence of offences under the Ac

##### 3.5.3.3. Police Stop and Search Values Check

In [21]:
police_stop_search_df.describe().show()

+-------+--------------------+------------------+-------------------+--------------------+------+---------+----------------------+-------------------------+--------------------+--------------------+--------------------+
|summary|                Type|Policing operation|           Latitude|           Longitude|Gender|Age range|Self-defined ethnicity|Officer-defined ethnicity|         Legislation|    Object of search|             Outcome|
+-------+--------------------+------------------+-------------------+--------------------+------+---------+----------------------+-------------------------+--------------------+--------------------+--------------------+
|  count|              402848|                 0|             376732|              376732|386257|   350606|                397337|                   384721|              402323|              401517|              402841|
|   mean|                NULL|              NULL| 51.511015042818386|-0.10028532732817852|  NULL|     NULL|             

##### 3.5.3.4. Fill missing coordinates with readable placeholders.


In [22]:
# Fill all missing values in police_stop_search_df with None
police_stop_search_df = police_stop_search_df.fillna("None")

# Detect numeric columns automatically
numeric_cols_stop = [
    f.name for f in police_stop_search_df.schema.fields
    if isinstance(f.dataType, NumericType)
]

# Fill all numeric columns with 0.0
if numeric_cols_stop:
    police_stop_search_df = police_stop_search_df.fillna(0.0, subset=numeric_cols_stop)

police_stop_search_df.describe().show()

+-------+--------------------+------------------+------------------+--------------------+------+---------+----------------------+-------------------------+--------------------+--------------------+--------------------+
|summary|                Type|Policing operation|          Latitude|           Longitude|Gender|Age range|Self-defined ethnicity|Officer-defined ethnicity|         Legislation|    Object of search|             Outcome|
+-------+--------------------+------------------+------------------+--------------------+------+---------+----------------------+-------------------------+--------------------+--------------------+--------------------+
|  count|              402848|            402848|            402848|              402848|402848|   402848|                402848|                   402848|              402848|              402848|              402848|
|   mean|                NULL|              NULL| 48.17163724062439|-0.09378398784404875|  NULL|     NULL|                  

#### 3.5.4. Police Street Data Check

##### 3.5.4.1. Police Street Schema Check

In [23]:
print("Police Stop and Search Schema:")
police_street_df.printSchema()

Police Stop and Search Schema:
root
 |-- Crime ID: string (nullable = true)
 |-- Month: timestamp (nullable = true)
 |-- Reported by: string (nullable = true)
 |-- Falls within: string (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- LSOA code: string (nullable = true)
 |-- LSOA name: string (nullable = true)
 |-- Crime type: string (nullable = true)
 |-- Last outcome category: string (nullable = true)
 |-- Context: string (nullable = true)



##### 3.5.4.2. Police Street Data Rows Check

In [24]:
print("First 5 Records")
police_street_df.orderBy("Month").show(5)

print("Last 5 Records")
police_street_df.orderBy("Month").tail(5)

First 5 Records


+--------------------+-------------------+--------------------+--------------------+---------+---------+--------------------+---------+--------------------+--------------------+---------------------+-------+
|            Crime ID|              Month|         Reported by|        Falls within|Longitude| Latitude|            Location|LSOA code|           LSOA name|          Crime type|Last outcome category|Context|
+--------------------+-------------------+--------------------+--------------------+---------+---------+--------------------+---------+--------------------+--------------------+---------------------+-------+
|aaa04506e8a99561f...|2022-10-01 00:00:00|Metropolitan Poli...|Metropolitan Poli...|-1.196475|53.031455|On or near Winifr...|E01027931|       Ashfield 013B|Violence and sexu...| Status update una...|   NULL|
|                NULL|2022-10-01 00:00:00|Metropolitan Poli...|Metropolitan Poli...| 0.140576|51.583419|On or near Rams G...|E01000027|Barking and Dagen...|Anti-social 

[Row(Crime ID='6000271c1141339bbc6f5c91179e5d8cdacd4b9f5202af85e7024aa5b7c8c9ea', Month=datetime.datetime(2025, 9, 1, 0, 0), Reported by='Metropolitan Police Service', Falls within='Metropolitan Police Service', Longitude=-2.108864, Latitude=52.590254, Location='On or near Clover Ley', LSOA code='E01010473', LSOA name='Wolverhampton 015C', Crime type='Violence and sexual offences', Last outcome category='Under investigation', Context=None),
 Row(Crime ID='3f884dc0a9b2bf468a511cc511fdbb3fcc45ccc000847674aba52daaad01441e', Month=datetime.datetime(2025, 9, 1, 0, 0), Reported by='Metropolitan Police Service', Falls within='Metropolitan Police Service', Longitude=-2.135378, Latitude=52.591332, Location='On or near Paget Street', LSOA code='E01010529', LSOA name='Wolverhampton 016D', Crime type='Other theft', Last outcome category='Under investigation', Context=None),
 Row(Crime ID='47294e07ff6cec5b704cc774fb66ecc0e84381a52455ac15684ffafc93ccd0e5', Month=datetime.datetime(2025, 9, 1, 0, 0), 

##### 3.5.4.3. Police Street Values Check

In [25]:
police_street_df.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------+---------+--------------------+---------------------+-------+
|summary|            Crime ID|         Reported by|        Falls within|           Longitude|           Latitude|            Location|LSOA code|LSOA name|          Crime type|Last outcome category|Context|
+-------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+---------+---------+--------------------+---------------------+-------+
|  count|             2714412|             3415444|             3415444|             3384154|            3384154|             3415444|  3384153|  3384153|             3415444|              2714412|      0|
|   mean|                NULL|                NULL|                NULL|-0.12138675667094415|  51.51239091004543|                NULL|     NULL|     NULL|                NULL| 

##### 3.5.4.4. Fill missing coordinates with readable placeholders.


In [26]:
# Fill all missing values in police_street_df with None
police_street_df = police_street_df.fillna("None")

# Detect numeric columns automatically
numeric_cols_street = [
    f.name for f in police_street_df.schema.fields
    if isinstance(f.dataType, NumericType)
]

# Fill all numeric columns with 0.0
if numeric_cols_street:
    police_street_df = police_street_df.fillna(0.0, subset=numeric_cols_street)

police_street_df.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+---------+---------+--------------------+---------------------+-------+
|summary|            Crime ID|         Reported by|        Falls within|           Longitude|         Latitude|            Location|LSOA code|LSOA name|          Crime type|Last outcome category|Context|
+-------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+---------+---------+--------------------+---------------------+-------+
|  count|             3415444|             3415444|             3415444|             3415444|          3415444|             3415444|  3415444|  3415444|             3415444|              3415444|3415444|
|   mean|                NULL|                NULL|                NULL|-0.12027469287594887|51.04046904232477|                NULL|     NULL|     NULL|                NULL|           

## 4. Data Transformation

In [27]:
os.makedirs("Datasets/Cleaned-Data/Police-Data", exist_ok=True)
os.makedirs("Datasets/Cleaned-Data/Income-Data", exist_ok=True)
os.makedirs("Datasets/Cleaned-Data/Population-Data", exist_ok=True)
os.makedirs("Datasets/Cleaned-Data/Map-Data", exist_ok=True)

NameError: name 'os' is not defined

### 4.1. Transform Cleaned Income Dataset

In [28]:
total_income_df.show(5)

+---------+-----------------+--------------------+--------------------+-----------+-----------+-----------------------+--------------------------+--------------------------+-----------------------+
|MSOA code|        MSOA name|Local authority code|Local authority name|Region code|Region name|Total annual income (£)|Upper confidence limit (£)|Lower confidence limit (£)|Confidence interval (£)|
+---------+-----------------+--------------------+--------------------+-----------+-----------+-----------------------+--------------------------+--------------------------+-----------------------+
|E02004297|County Durham 001|           E06000047|       County Durham|  E12000001| North East|                  41400|                     49100|                     34900|                  14200|
|E02004290|County Durham 002|           E06000047|       County Durham|  E12000001| North East|                  41100|                     48800|                     34700|                  14100|
|E02004298

In [29]:
# Select only the required columns from clean_df
selected_income_df = total_income_df.select(
    "MSOA code",
    "MSOA name",
    "Total annual income (£)"
)

selected_income_df.show(5)

+---------+-----------------+-----------------------+
|MSOA code|        MSOA name|Total annual income (£)|
+---------+-----------------+-----------------------+
|E02004297|County Durham 001|                  41400|
|E02004290|County Durham 002|                  41100|
|E02004298|County Durham 003|                  44300|
|E02004299|County Durham 004|                  35400|
|E02004291|County Durham 005|                  34500|
+---------+-----------------+-----------------------+
only showing top 5 rows


In [30]:
def simplify_column_names(df):
    new_cols = [col.lower().replace(" ", "_") for col in df.columns]
    return df.toDF(*new_cols)

selected_income_df = simplify_column_names(selected_income_df)
selected_income_df.show(5)

+---------+-----------------+-----------------------+
|msoa_code|        msoa_name|total_annual_income_(£)|
+---------+-----------------+-----------------------+
|E02004297|County Durham 001|                  41400|
|E02004290|County Durham 002|                  41100|
|E02004298|County Durham 003|                  44300|
|E02004299|County Durham 004|                  35400|
|E02004291|County Durham 005|                  34500|
+---------+-----------------+-----------------------+
only showing top 5 rows


In [31]:
selected_income_df.printSchema()

root
 |-- msoa_code: string (nullable = true)
 |-- msoa_name: string (nullable = true)
 |-- total_annual_income_(£): integer (nullable = true)



In [32]:
# Save the cleaned income data
selected_income_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Income-Data/")

### 4.2. Transform Cleaned Population Dataset

In [33]:
population_2022_df.show(5)
population_2023_df.show(5)
population_2024_df.show(5)

+-------------+-------------+--------------+---------------+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|LAD 2023 Code|LAD 2023 Name|LSOA 2021 Code| LSOA 2021 Name|Total| F0| F1| F2| F3| F4| F5| F6| F7| F8| F9|F10|F11|F12|F13|F14|F15|F16|F17|F18|F19|F20|F21|F22|F23|F24|F25|F26|F27|F28|F29|F30|F31|F32|F33|F34

In [34]:
def select_population_columns(population_df):
    # Select only the required columns from the population DataFrame
    return population_df.select(
        "LSOA 2021 Code",
        "LSOA 2021 Name",
        "Total"

    )

selected_population_2022_df = select_population_columns(population_2022_df)
selected_population_2023_df = select_population_columns(population_2023_df)
selected_population_2024_df = select_population_columns(population_2024_df)

selected_population_2022_df.show(5)
selected_population_2023_df.show(5)
selected_population_2024_df.show(5)


+--------------+---------------+-----+
|LSOA 2021 Code| LSOA 2021 Name|Total|
+--------------+---------------+-----+
|     E01011949|Hartlepool 009A| 1876|
|     E01011950|Hartlepool 008A| 1117|
|     E01011951|Hartlepool 007A| 1260|
|     E01011952|Hartlepool 002A| 1635|
|     E01011953|Hartlepool 002B| 1984|
+--------------+---------------+-----+
only showing top 5 rows
+--------------+---------------+-----+
|LSOA 2021 Code| LSOA 2021 Name|Total|
+--------------+---------------+-----+
|     E01011949|Hartlepool 009A| 1925|
|     E01011950|Hartlepool 008A| 1177|
|     E01011951|Hartlepool 007A| 1320|
|     E01011952|Hartlepool 002A| 1670|
|     E01011953|Hartlepool 002B| 2075|
+--------------+---------------+-----+
only showing top 5 rows


+--------------+---------------+-----+
|LSOA 2021 Code| LSOA 2021 Name|Total|
+--------------+---------------+-----+
|     E01011949|Hartlepool 009A| 1898|
|     E01011950|Hartlepool 008A| 1247|
|     E01011951|Hartlepool 007A| 1393|
|     E01011952|Hartlepool 002A| 1669|
|     E01011953|Hartlepool 002B| 2303|
+--------------+---------------+-----+
only showing top 5 rows


In [35]:
def simplify_column_names(df):
    new_cols = [col.lower().replace(" ", "_") for col in df.columns]
    return df.toDF(*new_cols)

selected_population_2022_df = simplify_column_names(selected_population_2022_df)
selected_population_2022_df.show(5)

selected_population_2023_df = simplify_column_names(selected_population_2023_df)
selected_population_2023_df.show(5)

selected_population_2024_df = simplify_column_names(selected_population_2024_df)
selected_population_2024_df.show(5)

+--------------+---------------+-----+
|lsoa_2021_code| lsoa_2021_name|total|
+--------------+---------------+-----+
|     E01011949|Hartlepool 009A| 1876|
|     E01011950|Hartlepool 008A| 1117|
|     E01011951|Hartlepool 007A| 1260|
|     E01011952|Hartlepool 002A| 1635|
|     E01011953|Hartlepool 002B| 1984|
+--------------+---------------+-----+
only showing top 5 rows
+--------------+---------------+-----+
|lsoa_2021_code| lsoa_2021_name|total|
+--------------+---------------+-----+
|     E01011949|Hartlepool 009A| 1925|
|     E01011950|Hartlepool 008A| 1177|
|     E01011951|Hartlepool 007A| 1320|
|     E01011952|Hartlepool 002A| 1670|
|     E01011953|Hartlepool 002B| 2075|
+--------------+---------------+-----+
only showing top 5 rows
+--------------+---------------+-----+
|lsoa_2021_code| lsoa_2021_name|total|
+--------------+---------------+-----+
|     E01011949|Hartlepool 009A| 1898|
|     E01011950|Hartlepool 008A| 1247|
|     E01011951|Hartlepool 007A| 1393|
|     E01011952|

In [36]:
selected_population_2022_df.printSchema()
selected_population_2023_df.printSchema()
selected_population_2024_df.printSchema()

root
 |-- lsoa_2021_code: string (nullable = true)
 |-- lsoa_2021_name: string (nullable = true)
 |-- total: integer (nullable = true)

root
 |-- lsoa_2021_code: string (nullable = true)
 |-- lsoa_2021_name: string (nullable = true)
 |-- total: integer (nullable = true)

root
 |-- lsoa_2021_code: string (nullable = true)
 |-- lsoa_2021_name: string (nullable = true)
 |-- total: integer (nullable = true)



In [37]:
# Save the cleaned income data
selected_population_2022_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Population-Data/2022")
selected_population_2023_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Population-Data/2023")
selected_population_2024_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Population-Data/2024")

### 4.3. Transform Cleaned Map Dataset

In [38]:
map_lsoa_df.show(5)
map_msoa_df.show(5)

+---+---------+---------+-------------------+---------+------+------+--------+--------+----------------+----------------+--------------------+
|FID|   OA21CD| LSOA21CD|           LSOA21NM|LSOA21NMW| BNG_E| BNG_N|     LAT|    LONG|     Shape__Area|   Shape__Length|            GlobalID|
+---+---------+---------+-------------------+---------+------+------+--------+--------+----------------+----------------+--------------------+
|  1|E00000001|E01000001|City of London 001A|         |532250|181864|51.52022|-0.09523|6949.15148162842|421.166161134957|3a44dd3d-5082-4a0...|
|  2|E00000003|E01000001|City of London 001A|         |532171|181819|51.51983|-0.09638|4492.41107177734|307.714652773369|f1216dc8-14d1-485...|
|  3|E00000005|E01000001|City of London 001A|         |532166|181722|51.51896|-0.09649|8565.51421356201|385.204781363993|44d6f70f-549c-428...|
|  4|E00000007|E01000001|City of London 001A|         |532088|181473|51.51674|-0.09771|75994.8297042847|1408.60765694562|4dd683e1-9a5c-46c...|

In [39]:
def select_lsoa_columns(map_lsoa_df):
    # Select only the required columns from the population DataFrame
    return map_lsoa_df.select(
        "LSOA21CD",
        "LSOA21NM",
        "LAT",
        "LONG"
    )

selected_map_lsoa_df = select_lsoa_columns(map_lsoa_df)

selected_map_lsoa_df.show(5)

+---------+-------------------+--------+--------+
| LSOA21CD|           LSOA21NM|     LAT|    LONG|
+---------+-------------------+--------+--------+
|E01000001|City of London 001A|51.52022|-0.09523|
|E01000001|City of London 001A|51.51983|-0.09638|
|E01000001|City of London 001A|51.51896|-0.09649|
|E01000001|City of London 001A|51.51674|-0.09771|
|E01000003|City of London 001C| 51.5225|-0.09741|
+---------+-------------------+--------+--------+
only showing top 5 rows


In [40]:
def select_msoa_columns(map_msoa_df):
    # Select only the required columns from the population DataFrame
    return map_msoa_df.select(
        "MSOA21CD",
        "MSOA21NM",
        "LAT",
        "LONG"
    )

selected_map_msoa_df = select_msoa_columns(map_msoa_df)

selected_map_msoa_df.show(5)

+---------+--------------------+--------+--------+
| MSOA21CD|            MSOA21NM|     LAT|    LONG|
+---------+--------------------+--------+--------+
|E02000001|  City of London 001|51.51562|-0.09349|
|E02000002|Barking and Dagen...|51.58652|0.138756|
|E02000003|Barking and Dagen...|51.57606|0.138149|
|E02000004|Barking and Dagen...|51.55639|0.176828|
|E02000005|Barking and Dagen...|51.56069|0.144267|
+---------+--------------------+--------+--------+
only showing top 5 rows


In [41]:
def simplify_column_names(df):
    new_cols = [col.lower().replace(" ", "_") for col in df.columns]
    return df.toDF(*new_cols)

selected_map_lsoa_df = simplify_column_names(selected_map_lsoa_df)
selected_map_lsoa_df.show(5)

selected_map_msoa_df = simplify_column_names(selected_map_msoa_df)
selected_map_msoa_df.show(5)


+---------+-------------------+--------+--------+
| lsoa21cd|           lsoa21nm|     lat|    long|
+---------+-------------------+--------+--------+
|E01000001|City of London 001A|51.52022|-0.09523|
|E01000001|City of London 001A|51.51983|-0.09638|
|E01000001|City of London 001A|51.51896|-0.09649|
|E01000001|City of London 001A|51.51674|-0.09771|
|E01000003|City of London 001C| 51.5225|-0.09741|
+---------+-------------------+--------+--------+
only showing top 5 rows
+---------+--------------------+--------+--------+
| msoa21cd|            msoa21nm|     lat|    long|
+---------+--------------------+--------+--------+
|E02000001|  City of London 001|51.51562|-0.09349|
|E02000002|Barking and Dagen...|51.58652|0.138756|
|E02000003|Barking and Dagen...|51.57606|0.138149|
|E02000004|Barking and Dagen...|51.55639|0.176828|
|E02000005|Barking and Dagen...|51.56069|0.144267|
+---------+--------------------+--------+--------+
only showing top 5 rows


In [42]:
selected_map_lsoa_df.printSchema()
selected_map_msoa_df.printSchema()

root
 |-- lsoa21cd: string (nullable = true)
 |-- lsoa21nm: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)

root
 |-- msoa21cd: string (nullable = true)
 |-- msoa21nm: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)



In [43]:
# Save the cleaned income data
selected_map_lsoa_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Map-Data/lsoa")
selected_map_msoa_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Map-Data/msoa")
    

### 4.4. Transform Cleaned Police Dataset

In [44]:
police_outcomes_df.show(5)
police_stop_search_df.show(5)
police_street_df.show(5)

+--------------------+-------------------+--------------------+--------------------+---------+---------+--------------------+---------+--------------+--------------------+
|            Crime ID|              Month|         Reported by|        Falls within|Longitude| Latitude|            Location|LSOA code|     LSOA name|        Outcome type|
+--------------------+-------------------+--------------------+--------------------+---------+---------+--------------------+---------+--------------+--------------------+
|2551bf97ea2627325...|2023-10-01 00:00:00|Metropolitan Poli...|Metropolitan Poli...|      0.0|      0.0|         No location|     None|          None|Investigation com...|
|2c6e4b41131baf4a5...|2023-10-01 00:00:00|Metropolitan Poli...|Metropolitan Poli...|      0.0|      0.0|         No location|     None|          None|Investigation com...|
|7c6c6ad4d0a255ba0...|2023-10-01 00:00:00|Metropolitan Poli...|Metropolitan Poli...|      0.0|      0.0|         No location|     None|     

In [45]:
def simplify_column_names(df):
    new_cols = [col.lower().replace(" ", "_") for col in df.columns]
    return df.toDF(*new_cols)

police_outcomes_df = simplify_column_names(police_outcomes_df)
police_outcomes_df.show(5)

police_stop_search_df = simplify_column_names(police_stop_search_df)
police_stop_search_df.show(5)

police_street_df = simplify_column_names(police_street_df)
police_street_df.show(5)

+--------------------+-------------------+--------------------+--------------------+---------+---------+--------------------+---------+--------------+--------------------+
|            crime_id|              month|         reported_by|        falls_within|longitude| latitude|            location|lsoa_code|     lsoa_name|        outcome_type|
+--------------------+-------------------+--------------------+--------------------+---------+---------+--------------------+---------+--------------+--------------------+
|2551bf97ea2627325...|2023-10-01 00:00:00|Metropolitan Poli...|Metropolitan Poli...|      0.0|      0.0|         No location|     None|          None|Investigation com...|
|2c6e4b41131baf4a5...|2023-10-01 00:00:00|Metropolitan Poli...|Metropolitan Poli...|      0.0|      0.0|         No location|     None|          None|Investigation com...|
|7c6c6ad4d0a255ba0...|2023-10-01 00:00:00|Metropolitan Poli...|Metropolitan Poli...|      0.0|      0.0|         No location|     None|     

In [46]:
# Save the cleaned income data
# police_stop_search_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Police-Data/stop-search")
# police_outcomes_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Police-Data/outcomes")
# police_street_df.coalesce(1).write.mode("overwrite").option("header", "true").csv("Datasets/Cleaned-Data/Police-Data/street")

In [47]:
# Save stop & search data as Parquet
police_stop_search_df.write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("Datasets/Cleaned-Data/Police-Data/stop-search-parquet")

# Save outcomes data as Parquet
police_outcomes_df.write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("Datasets/Cleaned-Data/Police-Data/outcomes-parquet")

# Save street data as Parquet
police_street_df.write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet("Datasets/Cleaned-Data/Police-Data/street-parquet")


## 5. Data Querying